In [3]:
# Import Libraries
import pandas as pd
from nltk.corpus import stopwords
import requests
import pyterrier as pt
from Preprocessing.Preprocess import split_by_language,compute_similarity,convert2json
if not pt.started():
    pt.init()

C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_2984\1770673465.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [4]:

### Load The Data From The APIs

# Get Lawyers From Lawyers API
lawyer_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/lawyers")
print(f"status code : {lawyer_response.status_code}")


# Get Rates From Rates API 
rate_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/rates")
print(f"status code : {rate_response.status_code}")


# Get Agencies From Agencies API
agency_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/agencies")
print(f"status code : {agency_response.status_code}")


# Get Issues From Issues API
issue_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/issues")
print(f"status code : {issue_response.status_code}")


status code : 200
status code : 200
status code : 200
status code : 200


In [61]:
### Lawyers Data Perprocessing
lawyers_response = lawyer_response.json()
lawyers_pere = pd.DataFrame(lawyers_response['lawyers'])
lawyers = lawyers_pere.rename(columns={
    "id":"lawyer_id"
}).drop(["name","email","union_number","affiliation_date","phone","rates","avatar"],axis=1)
lawyers.head()

,lawyer_id,address,union_branch,specializations,years_of_experience,description,rank
0,1,94949 ممر مالك العقل شقة رقم. 07\nالباحة,مكة,"[Civil, Administrative, Military, Criminal, In...",24,يتشفه به أو يدافع عن رقبته، وهي كلها ظلمات بعض...,4.00
1,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67
2,3,33307 طريق فجر السماري\nالرس,الطائف,"[Civil, International Disputes, Administrative...",39,ما أودعه هذه الاوراق فان المجال ضيق، والتحكم ب...,3.00
3,4,78255 طريق رزق الفدا شقة رقم. 62\nسيهات,المدينة المنورة,"[Terrorism, Military, Criminal, Administrative...",6,شعر بالموجود الواجب الوجود، وهذا الشيء العارف،...,3.00
4,5,8183 شارع سارة الفريدي\nمكة,جيزان,"[Legal, International Disputes, Criminal, Terr...",22,للحيوان والنبات. ثم ينظر إلى أذنيها والى عينيه...,3.75


In [62]:
rates_response = rate_response.json()
rates = pd.DataFrame(rates_response["rates"])
rates.head()

,id,user_id,lawyer_id,rating,review
0,1,12,2,3,أغلظ الفضلتين فبالاذناب، وأما مخرج وأما مخرج و...
1,2,24,22,2,الذي ليس بجسم، ولا يحتاج منه في تسيير الجبال و...
2,3,40,39,3,أراد أن يعلم بأي شيء حصل له العلم بهذا الموجود...
3,4,3,3,2,وهذا المحدث الذي أحدثه، لم أحدثه الآن ولم يحدث...
4,5,43,28,2,وشقاوةً أطم ممن إذا تصفحت أعماله من وقت انتباه...


In [2]:
### Rates Data Preproceesing
rates_response = rate_response.json()
rates = pd.DataFrame(rates_response["rates"])
rates.dropna(inplace=True)
rates.drop(["id"],axis=1,inplace=True)
# rates['rating'] = rates['rating'].apply(lambda x :f"{x}star")
rates.head()

NameError: name 'rate_response' is not defined

In [37]:
### Agencies Data Perprcessing
agencies_response = agency_response.json()
agencies = pd.DataFrame(agencies_response["agencies"]).rename(columns={
    "id":"agency_id"
})
agencies = agencies[["agency_id","lawyer_id"]]
agencies.head()

,agency_id,lawyer_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [64]:
### Issues Data Preprocessing
issues_response = issue_response.json()
issues = pd.DataFrame(issues_response["issues"]).drop(["base_number","record_number","id","start_date","end_date","status"],axis=1)
issues.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active
0,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No
1,96,72067044,31458938,2,Civil,4,Appeal,6741509,No
2,14,02242369,75774024,2,Civil,5,Commercial,23226642,No
3,33,01280952,00767819,3,Criminal,11,Criminals,21113327,No
4,20,48282405,09148390,4,Legitimacy,12,Islamic,24534061,No


In [65]:
### Merge Lawyers With Rates
lawyers_with_rates = lawyers.merge(rates,on="lawyer_id")
lawyers_with_rates.head()


,lawyer_id,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating,review
0,1,94949 ممر مالك العقل شقة رقم. 07\nالباحة,مكة,"[Civil, Administrative, Military, Criminal, In...",24,يتشفه به أو يدافع عن رقبته، وهي كلها ظلمات بعض...,4.00,12,5,وران على قلوبهم ما يكسبون ختم الله على يديه طا...
1,1,94949 ممر مالك العقل شقة رقم. 07\nالباحة,مكة,"[Civil, Administrative, Military, Criminal, In...",24,يتشفه به أو يدافع عن رقبته، وهي كلها ظلمات بعض...,4.00,42,3,والنمط الذي كلامنا فيه فوق هذا كله، ولم يكن يد...
2,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67,12,3,أغلظ الفضلتين فبالاذناب، وأما مخرج وأما مخرج و...
3,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67,43,2,الموانع الأرضية، فلقولهم: أن الإقليم الرابع اع...
4,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67,3,3,تثبيته، فلم أفعل ذلك إلا نبوأً ونفاراً، مع أنه...


In [66]:
### Merge Lawyers_with_rates with Agencies
lawyers_with_rates= pd.merge(agencies,lawyers_with_rates,on=["lawyer_id"],how="inner")
lawyers_with_rates.head()

,agency_id,lawyer_id,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating,review
0,1,1,94949 ممر مالك العقل شقة رقم. 07\nالباحة,مكة,"[Civil, Administrative, Military, Criminal, In...",24,يتشفه به أو يدافع عن رقبته، وهي كلها ظلمات بعض...,4.00,12,5,وران على قلوبهم ما يكسبون ختم الله على يديه طا...
1,1,1,94949 ممر مالك العقل شقة رقم. 07\nالباحة,مكة,"[Civil, Administrative, Military, Criminal, In...",24,يتشفه به أو يدافع عن رقبته، وهي كلها ظلمات بعض...,4.00,42,3,والنمط الذي كلامنا فيه فوق هذا كله، ولم يكن يد...
2,2,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67,12,3,أغلظ الفضلتين فبالاذناب، وأما مخرج وأما مخرج و...
3,2,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67,43,2,الموانع الأرضية، فلقولهم: أن الإقليم الرابع اع...
4,2,2,32264 طريق عواد الحنتوشي شقة رقم. 19\nالقريات,الأفلاج,"[Criminal, Terrorism, International Disputes, ...",18,الأقصى إنما هو على سبيل الإضاءة لا غير، ولعل ت...,2.67,3,3,تثبيته، فلم أفعل ذلك إلا نبوأً ونفاراً، مع أنه...


In [68]:
### Merge Them With Issues
lawyers_with_rates = pd.merge(issues,lawyers_with_rates,on="agency_id",how="inner")
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,lawyer_id,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating,review
0,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,47,1,صفات الأجسام من الجمادات والأحياء، فرأى أن كل ...
1,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,52,3,الضباب الابيض، فأدخل إصبعه فيه، فوجده من الحرا...
2,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,29,4,الوجود؛ وان كانت تلك الصورة بحيث لا يعرض بطرفة...
3,14,02242369,75774024,2,Civil,5,Commercial,23226642,No,15,81 طريق عبد الحافظ القحطاني شقة رقم. 79\nضباء,الخرج,"[Arbitration, Terrorism, Commercial, Criminal]",12,الرفيع الثابت الوجود الذي لا سبب لتكون الحرارة...,2.00,49,2,فوق بعض في بحر لجي وان منكم إلا واردها كان على...
4,33,01280952,00767819,3,Criminal,11,Criminals,21113327,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,47,1,صفات الأجسام من الجمادات والأحياء، فرأى أن كل ...


In [69]:
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,lawyer_id,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating,review
0,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,47,1,صفات الأجسام من الجمادات والأحياء، فرأى أن كل ...
1,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,52,3,الضباب الابيض، فأدخل إصبعه فيه، فوجده من الحرا...
2,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,29,4,الوجود؛ وان كانت تلك الصورة بحيث لا يعرض بطرفة...
3,14,02242369,75774024,2,Civil,5,Commercial,23226642,No,15,81 طريق عبد الحافظ القحطاني شقة رقم. 79\nضباء,الخرج,"[Arbitration, Terrorism, Commercial, Criminal]",12,الرفيع الثابت الوجود الذي لا سبب لتكون الحرارة...,2.00,49,2,فوق بعض في بحر لجي وان منكم إلا واردها كان على...
4,33,01280952,00767819,3,Criminal,11,Criminals,21113327,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,47,1,صفات الأجسام من الجمادات والأحياء، فرأى أن كل ...


In [70]:
### Add Data To Text Column
lawyers_with_rates["Text"] = lawyers_with_rates[['court_name','address','union_branch','estimated_cost']].astype(str).agg(" ".join,axis=1)
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,lawyer_id,address,union_branch,specializations,years_of_experience,description,rank,user_id,rating,review,Text
0,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,47,1,صفات الأجسام من الجمادات والأحياء، فرأى أن كل ...,Legitimacy 9103 ممر سيرينا الصقير\nالحوية الزل...
1,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,52,3,الضباب الابيض، فأدخل إصبعه فيه، فوجده من الحرا...,Legitimacy 9103 ممر سيرينا الصقير\nالحوية الزل...
2,33,01280952,00767819,4,Legitimacy,13,Christianity,16549600,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,29,4,الوجود؛ وان كانت تلك الصورة بحيث لا يعرض بطرفة...,Legitimacy 9103 ممر سيرينا الصقير\nالحوية الزل...
3,14,02242369,75774024,2,Civil,5,Commercial,23226642,No,15,81 طريق عبد الحافظ القحطاني شقة رقم. 79\nضباء,الخرج,"[Arbitration, Terrorism, Commercial, Criminal]",12,الرفيع الثابت الوجود الذي لا سبب لتكون الحرارة...,2.00,49,2,فوق بعض في بحر لجي وان منكم إلا واردها كان على...,Civil 81 طريق عبد الحافظ القحطاني شقة رقم. 79\...
4,33,01280952,00767819,3,Criminal,11,Criminals,21113327,No,36,9103 ممر سيرينا الصقير\nالحوية,الزلفي,"[International Disputes, Arbitration, Civil, T...",29,ظهورهم، واشتروا بها ثمناً قليلاً، وألهاهم عن ذ...,2.67,47,1,صفات الأجسام من الجمادات والأحياء، فرأى أن كل ...,Criminal 9103 ممر سيرينا الصقير\nالحوية الزلفي...


In [71]:
### Text Data Preprocessing
lawyers_with_rates['processed_text'] = lawyers_with_rates['Text'].apply(split_by_language)
# lawyers_with_rates.head()

In [72]:
results = compute_similarity(lawyers_with_rates,rates['review'][0])
# print(results.values)

In [73]:

filtered_df = lawyers_pere[lawyers_pere['id'].isin(results.values)]
# filtered_df.head()

In [74]:
# Convert to JSON format
results = {
        "isSuccess": True,
        "lawyers": lawyers_pere.to_dict(orient="records")  # Convert DataFrame to list of dictionaries
    }
results

{'isSuccess': True,
 'lawyers': [{'id': 1,
   'name': 'سوار جبير الفدا',
   'email': 'effertz.bud@example.com',
   'address': '94949 ممر مالك العقل شقة رقم. 07\nالباحة',
   'union_branch': 'مكة',
   'union_number': '12156905',
   'affiliation_date': '1979-05-05',
   'specializations': ['Civil',
    'Administrative',
    'Military',
    'Criminal',
    'International Disputes',
    'Terrorism',
    'Arbitration',
    'Commercial'],
   'years_of_experience': 24,
   'description': 'يتشفه به أو يدافع عن رقبته، وهي كلها ظلمات بعضها فوق بعض في بحر لجي وان منكم إلا واردها كان على ربك حتماً مقضياً. فلما فهم أحوال الناس وان أكثرهم بمنزلة الحيوان غير الناطق علم أن ذاته التي أدركه بها.',
   'phone': '(214) 239-4823',
   'avatar': 'https://via.placeholder.com/640x480.png/007711?text=rerum',
   'rates': [{'id': 19,
     'user_id': 12,
     'lawyer_id': 1,
     'rating': 5,
     'review': 'وران على قلوبهم ما يكسبون ختم الله على يديه طائفة من تلك الأشياء الآخر التي يكون فسادها سبباً.'},
    {'id': 99